## Imports and initial config

In [1]:
# %pip3 install --upgrade --quiet google-cloud-storage

In [2]:
# %pip install transformers datasets evaluate accelerate

In [3]:
# %pip install torch

In [4]:
# # restart the kernel to get the newly installed packages

# import IPython


# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

In [5]:
# # Importing helper libraries
# import ipywidgets as widgets
# import json

**- Authenticate code and libraries with your credentials :**


In [6]:
# ! gcloud auth application-default login

**- Set a project if differs from default :**

In [7]:
# PROJECT_ID = "genai-l400-labs"  # @param {type:"string"}

# # Set the project id
# ! gcloud config set project {PROJECT_ID}

#### Imports and initialize the model

In [8]:
# Cell 1: Import required libraries
import torch
from transformers import pipeline
from IPython.display import clear_output, display
import ipywidgets as widgets
import json
import sys
import os

# # Add the current directory to Python path if not already there
# if os.getcwd() not in sys.path:
#     sys.path.append(os.getcwd())

# Import helper functions
from utils import (generate_proposal_section, create_section_heading, 
                  create_text_area, create_button, generate_html_proposal,
                  save_and_download_html)

In [9]:
# you need to have access to this gated model onhuggingface
# run 'huggingface-cli login' in the terminal

In [10]:

pipe = pipeline("text-generation", "meta-llama/Meta-Llama-3.1-8B-Instruct", 
torch_dtype=torch.bfloat16, device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cpu


## Main Code

In [11]:
#Collect essential base information for the sales proposal.

def base_information_step(proposal_data, next_callback):
    clear_output(wait=True)
    display(create_section_heading("Step 1: Base Information"))
    
    title_input = widgets.Text(description="Title:", placeholder="e.g., Cloud Migration Proposal")
    date_input = widgets.Text(description="Date:", placeholder="e.g., October 15, 2023")
    customer_input = widgets.Text(description="Customer:", placeholder="e.g., Acme Corporation")
    salesperson_input = widgets.Text(description="Salesperson:", placeholder="e.g., Jane Smith")
    

    display(title_input, date_input, customer_input, salesperson_input)
    
    next_button = create_button("Next Step")
    display(next_button)
    
    def on_next_button_clicked(b):
        proposal_data['title'] = title_input.value
        proposal_data['date'] = date_input.value
        proposal_data['customer'] = customer_input.value
        proposal_data['salesperson'] = salesperson_input.value
        
        next_callback()
    
    next_button.on_click(on_next_button_clicked)

In [12]:
# Executive Summary generation
def executive_summary_step(pipe, proposal_data, next_callback):
    clear_output(wait=True)
    display(create_section_heading("Step 2: Executive Summary"))
    
    # Create input widget for executive summary context
    context_input = create_text_area(
        "Context:", 
        "Any additional context about the project, customer and your relationship with them"
    )
    display(context_input)
    
    # Create generation button
    generate_button = create_button("Generate Executive Summary")
    display(generate_button)
    
    # Create output area
    summary_output = widgets.Output()
    display(summary_output)
    
    # Text area for editing (initially hidden)
    summary_text = create_text_area("Edit:", "", width='90%', height='200px')
    
    # Next button (initially hidden)
    next_step_button = create_button("Next Step")
    
    def on_generate_button_clicked(b):
        with summary_output:
            clear_output(wait=True)
            display(widgets.HTML("<p>Generating executive summary...</p>"))
            
            # Create prompt for executive summary
            prompt = f"""
            Create a  concise, compelling Executive Summary for a sales proposal with the following details:
            - Title: {proposal_data['title']}
            - Customer: {proposal_data['customer']}
            - Date: {proposal_data['date']}
            - Additional Context: {context_input.value}
            
            Infer the The executive summary should be concise (3-5 sentences), highlight the key value proposition,
            and create a compelling case for the customer to continue reading. 
            Be professional, confident, and focus on the customer's needs.
            """
            
            # Generate the executive summary
            executive_summary = generate_proposal_section(pipe, prompt)
            
            clear_output(wait=True)
            summary_text.value = executive_summary
            display(summary_text)
            display(next_step_button)
    
    generate_button.on_click(on_generate_button_clicked)
    
    def on_next_step_button_clicked(b):
        # Store the executive summary
        proposal_data['executive_summary'] = summary_text.value
        
        # Move to next step
        next_callback()
    
    next_step_button.on_click(on_next_step_button_clicked)

In [13]:
# Generate a Problem Statement
def problem_statement_step(pipe, proposal_data, next_callback):

    clear_output(wait=True)
    display(create_section_heading("Step 3: Problem Statement"))
    

    context_input = create_text_area(
        "Challenges:", 
        "Describe the customer's challenges, pain points, or business needs"
    )
    display(context_input)
    
  
    generate_button = create_button("Generate Problem Statement")
    display(generate_button)
    

    problem_output = widgets.Output()
    display(problem_output)
    
   
    problem_text = create_text_area("Edit:", "", width='90%', height='200px')
    

    next_step_button = create_button("Next Step")
    
    def on_generate_button_clicked(b):
        with problem_output:
            clear_output(wait=True)
            display(widgets.HTML("<p>Generating problem statement...</p>"))
            
 
            prompt = f"""
            Create a concise and clear problem statement for a sales proposal using the following details:
            - Customer: {proposal_data['customer']}
            - Executive Summary: {proposal_data['executive_summary']}
            - Challenges: {context_input.value}
            
            Carefully review the provided details. Your problem Statement should:
            1. Succinctly summarize the key issues the customer is facing, clearly connecting their identified challenges to the broader context presented in the Executive Summary. 
            2. Focus on accurately capturing the customer's situation
            3. Highlight essential pain points, and articulate why resolving these issues is critical.
            """
            
            problem_statement = generate_proposal_section(pipe, prompt)
            
            clear_output(wait=True)
            problem_text.value = problem_statement
            display(problem_text)
            display(next_step_button)
    
    generate_button.on_click(on_generate_button_clicked)
    
    def on_next_step_button_clicked(b):
        # Store the problem statement
        proposal_data['problem_statement'] = problem_text.value
        
        # Move to next step
        next_callback()
    
    next_step_button.on_click(on_next_step_button_clicked)

In [14]:
# Generate the proposed solution
def proposed_solution_step(pipe, proposal_data, next_callback):

    clear_output(wait=True)
    display(create_section_heading("Step 4: Proposed Solution"))
    

    solution_input = create_text_area(
        "Solution:", 
        "Briefly give details about your proposed solution, products, services, or approach offered",
        height='150px'
    )
    display(solution_input)
    
    # Create generation button
    generate_button = create_button("Generate Proposed Solution")
    display(generate_button)
    
    # Create output area
    solution_output = widgets.Output()
    display(solution_output)
    
    # Text area for editing (initially hidden)
    solution_text = create_text_area("Edit:", "", width='90%', height='300px')
    
    # Next button (initially hidden)
    next_step_button = create_button("Next Step")
    
    def on_generate_button_clicked(b):
        with solution_output:
            clear_output(wait=True)
            display(widgets.HTML("<p>Generating proposed solution...</p>"))
            
            # Create prompt for proposed solution
            prompt = f"""
            Create a detailed proposed solution for a sales proposal with the following details:
            - Customer: {proposal_data['customer']}
            - Problem Statement: {proposal_data['problem_statement']}
            - Solution Details: {solution_input.value}
            
            The proposed solution should:
            1. Expand on the solution details provided, describe the products, services, or approach being offered 
            2. Explain how each component addresses specific challenges mentioned in the Problem Statement
            3. Include proposed implementation details, phases, or timeline if applicable
            4. Highlight any unique or differentiating aspects of your solution
            
            Be specific, technical where appropriate, and focus on how your solution solves the customer's problems.
            """
            
            # Generate the proposed solution
            proposed_solution = generate_proposal_section(pipe, prompt, max_length=800)
            
            clear_output(wait=True)
            solution_text.value = proposed_solution
            display(solution_text)
            display(next_step_button)
    
    generate_button.on_click(on_generate_button_clicked)
    
    def on_next_step_button_clicked(b):
        # Store the proposed solution
        proposal_data['proposed_solution'] = solution_text.value
        
        # Move to next step
        next_callback()
    
    next_step_button.on_click(on_next_step_button_clicked)

In [15]:
# Generate the benefits section
def benefits_step(pipe, proposal_data, next_callback):

    clear_output(wait=True)
    display(create_section_heading("Step 5: Benefits"))
    

    benefits_input = create_text_area(
        "Additional Benefits:", 
        "Any specific benefits or ROI metrics you want to highlight"
    )
    display(benefits_input)
    

    generate_button = create_button("Generate Benefits")
    display(generate_button)
    

    benefits_output = widgets.Output()
    display(benefits_output)
    

    benefits_text = create_text_area("Edit:", "", width='90%', height='250px')
    

    next_step_button = create_button("Next Step")
    
    def on_generate_button_clicked(b):
        with benefits_output:
            clear_output(wait=True)
            display(widgets.HTML("<p>Generating benefits...</p>"))
            

            prompt = f"""
            Create a compelling benefits section for a sales proposal with the following details:
            - Customer: {proposal_data['customer']}
            - Problem Statement: {proposal_data['problem_statement']}
            - Proposed Solution: {proposal_data['proposed_solution']}
            - Additional Benefits: {benefits_input.value}
            
            The benefits section should:
            1. Clearly articulate the value and ROI of implementing the proposed solution
            2. Include both quantitative benefits (cost savings, efficiency gains, revenue increase) and qualitative benefits (improved reputation, customer satisfaction)
            3. Relate benefits directly to the customer's challenges and business goals
            4. Be specific and backed by data or examples where possible
            
            Format the benefits as bullet points or short paragraphs for easy readability.
            """
            

            benefits = generate_proposal_section(pipe, prompt)
            
            clear_output(wait=True)
            benefits_text.value = benefits
            display(benefits_text)
            display(next_step_button)
    
    generate_button.on_click(on_generate_button_clicked)
    
    def on_next_step_button_clicked(b):
     
        proposal_data['benefits'] = benefits_text.value
  
        next_callback()
    
    next_step_button.on_click(on_next_step_button_clicked)

In [16]:
# Generate pricing and terms
def pricing_terms_step(pipe, proposal_data, next_callback):

    clear_output(wait=True)
    display(create_section_heading("Step 6: Pricing and Terms"))
    

    pricing_input = create_text_area(
        "Pricing Details:", 
        "Enter pricing information, payment terms, etc.",
        height='150px'
    )
    display(pricing_input)
    

    generate_button = create_button("Generate Pricing and Terms")
    display(generate_button)
    

    pricing_output = widgets.Output()
    display(pricing_output)
    

    pricing_text = create_text_area("Edit:", "", width='90%', height='250px')
    

    next_step_button = create_button("Next Step")
    
    def on_generate_button_clicked(b):
        with pricing_output:
            clear_output(wait=True)
            display(widgets.HTML("<p>Generating pricing and terms...</p>"))
   
            prompt = f"""
            Create a clear pricing and terms section for a sales proposal with the following details:
            - Customer: {proposal_data['customer']}
            - Proposed Solution: {proposal_data['proposed_solution']}
            - Pricing Details: {pricing_input.value}
            
            The pricing and terms section should:
            1. Present the pricing structure in a clear, organized manner
            2. Include payment terms, schedule, and any financing options
            3. Specify what is included in the price and any additional costs
            4. Mention the validity period of the proposal
            5. Include any relevant legal terms or conditions
            
            Format the information in a professional, easy-to-read manner.
            """
            
            pricing_terms = generate_proposal_section(pipe, prompt)
            
            clear_output(wait=True)
            pricing_text.value = pricing_terms
            display(pricing_text)
            display(next_step_button)
    
    generate_button.on_click(on_generate_button_clicked)
    
    def on_next_step_button_clicked(b):
        proposal_data['pricing_terms'] = pricing_text.value 
        next_callback()
    
    next_step_button.on_click(on_next_step_button_clicked)

In [17]:
#Generate the conclusion section
def conclusion_step(pipe, proposal_data, next_callback):
    clear_output(wait=True)
    display(create_section_heading("Step 7: Conclusion"))
    
    # Create input widget for conclusion context
    conclusion_input = create_text_area(
        "Next Steps:", 
        "Describe desired next steps that you would like to include in the conclusion, if any"
    )
    display(conclusion_input)
    
    generate_button = create_button("Generate Conclusion")
    display(generate_button)
    
    conclusion_output = widgets.Output()
    display(conclusion_output)
    
    conclusion_text = create_text_area("Edit:", "", width='90%', height='200px')
    
    generate_final_button = create_button("Generate Final Proposal")
    
    def on_generate_button_clicked(b):
        with conclusion_output:
            clear_output(wait=True)
            display(widgets.HTML("<p>Generating conclusion...</p>"))
            
            # Create prompt for conclusion
            prompt = f"""
            Create a strong conclusion for a sales proposal with the following details:
            - Customer: {proposal_data['customer']}
            - Title: {proposal_data['title']}
            - Executive Summary: {proposal_data['executive_summary']}
            - Proposed Solution: {proposal_data['proposed_solution']}
            - Benefits: {proposal_data['benefits']}
            - Salesperson: {proposal_data['salesperson']}
            - Next Steps: {conclusion_input.value}
            
            The conclusion should:
            1. Summarize the key value proposition and benefits
            2. Express confidence in the solution's ability to address the customer's needs
            3. Include a clear call to action of next steps if any are given
            4. Thank the customer for their consideration
            5. Provide contact information and availability for questions
            
            End on a positive, confident note that encourages the customer to move forward.
            """
            
            conclusion = generate_proposal_section(pipe, prompt)
            
            clear_output(wait=True)
            conclusion_text.value = conclusion
            display(conclusion_text)
            display(generate_final_button)
    
    generate_button.on_click(on_generate_button_clicked)
    
    def on_generate_final_button_clicked(b):
        proposal_data['conclusion'] = conclusion_text.value
        
        next_callback()
    
    generate_final_button.on_click(on_generate_final_button_clicked)

In [18]:
# Compile the final proposal in HTML format
def final_proposal_step(proposal_data, restart_callback):

    clear_output(wait=True)
    display(create_section_heading("Final Proposal"))
    
    html_proposal = generate_html_proposal(proposal_data)
    
    display(widgets.HTML("<h3>HTML Preview:</h3>"))
    display(widgets.HTML(html_proposal))
    
    html_output = widgets.Textarea(
        value=html_proposal,
        description="HTML Code:",
        layout=widgets.Layout(width='90%', height='400px')
    )
    display(widgets.HTML("<h3>Raw HTML (Copy from here):</h3>"))
    display(html_output)
    
    download_button = create_button("Download HTML")
    display(download_button)
    
    def on_download_button_clicked(b):
        display(save_and_download_html(html_proposal))
    
    download_button.on_click(on_download_button_clicked)
    
    new_proposal_button = create_button("Create New Proposal", button_style='primary')
    display(new_proposal_button)
    
    new_proposal_button.on_click(lambda b: restart_callback())

In [21]:
# Main function for orchestration
def sales_proposal_wizard(pipe):
    """Create a sales proposal using a step-by-step wizard approach."""
    proposal_data = {}
    
    # Define the step sequence
    def run_executive_summary():
        executive_summary_step(pipe, proposal_data, run_problem_statement)
        
    def run_problem_statement():
        problem_statement_step(pipe, proposal_data, run_proposed_solution)
        
    def run_proposed_solution():
        proposed_solution_step(pipe, proposal_data, run_benefits)
        
    def run_benefits():
        benefits_step(pipe, proposal_data, run_pricing_terms)
        
    def run_pricing_terms():
        pricing_terms_step(pipe, proposal_data, run_conclusion)
        
    def run_conclusion():
        conclusion_step(pipe, proposal_data, run_final_proposal)
        
    def run_final_proposal():
        final_proposal_step(proposal_data, restart_wizard)
        
    def restart_wizard():
        clear_output(wait=True)
        sales_proposal_wizard(pipe)
    
    # Start with the first step
    base_information_step(proposal_data, run_executive_summary)

In [22]:
#Start the wizard
def start_wizard(pipe):
    """Display welcome message and start button for the wizard."""
    display(widgets.HTML("""
    <h1>AI-Powered Sales Proposal Generator</h1>
    <p>This wizard will guide you through creating a professional sales proposal step by step.</p>
    <p>Each section will be generated based on your input, and you'll have the opportunity to edit the content before moving to the next section.</p>
    <p>Click "Start" to begin building your proposal.</p>
    """))
    
    start_button = create_button("Start", button_style='success')
    display(start_button)
    
    def on_start_button_clicked(b):
        clear_output(wait=True)
        sales_proposal_wizard(pipe)
    
    start_button.on_click(on_start_button_clicked)

# Run the starter function
start_wizard(pipe)

HTML(value='<h2>Step 5: Benefits</h2>')

Textarea(value='', description='Additional Benefits:', layout=Layout(height='100px', width='80%'), placeholder…

Button(description='Generate Benefits', style=ButtonStyle())

Output()